## Benchmark Lorenz 96 linear and nonlinear filters

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [4]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using OrdinaryDiffEq
using JLD

In [5]:
using DelimitedFiles

Load some packages to make nice figures

In [6]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
# Plots.font("sans-serif")
# clibrary(:colorbrewer)
# gr()
pyplot()

using LaTeXStrings
# PyPlot.rc("text", usetex = "true")
# rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
# rcParams["text.usetex"] = true;
PyPlot.rc("font", family = "CMU Serif")
PyPlot.matplotlib[:rc]("mathtext",fontset="cm")        #computer modern font 
PyPlot.matplotlib[:rc]("font",family="serif",size=12)

using ColorSchemes

In [30]:
path = "/media/mat/HDD/AdaptiveTransportMap/notebooks/lorenz96/data/"

β_array = collect(0.95:0.01:1.05)
Ne_array = [10, 20, 40, 60, 100, 200, 400, 600]

model, data = setup_lorenz96(path, Ne_array);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Ne 10 RMSE: 0.6017422369576234


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Ne 20 RMSE: 0.5078680720281411


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


...

Ne 200 RMSE: 0.4871107090323029


Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


Ne 400 RMSE: 0.4948593933292423


Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


Ne 600 RMSE: 0.4949281341576929


In [31]:
save(path*"lorenz96_data.jld", "data", data)

Benchmark for the Lorenz-96 problem with the sequential stochastic EnkF

In [32]:
metric_enkf = benchmark_lorenz96(model, data, path, Ne_array, β_array);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.479686 seconds (3.51 M allocations: 331.776 MiB, 6.86% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.4063312146337485
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.481681 seconds (3.51 M allocations: 331.776 MiB, 5.96% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 3.031626178778555
(Ne, β) = (10, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:00:24


 24.598230 seconds (189.77 M allocations: 17.803 GiB, 8.97% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:17:12


Ne 600& β 1.05 RMSE: 0.49307306726541833


8-element Array{Any,1}:
 Metrics[Metrics(10, [0.32078591911714366, 0.4441365673287443, 0.6941005779109013, 0.8793263765117602, 0.8633711337936523, 0.6890628532888722, 0.5273699112797657, 0.4969749546922613, 0.73644871637577, 0.8104683288617239  …  0.31779182360591973, 0.23200415551482328, 0.26645162834570074, 0.5011198105086171, 0.2708562624664994, 0.12212871915966879, 0.023131665869033584, 0.32857741765535037, 1.0715025886964598, 1.1436552936346531], 0.5507688120244398, 1.4063312146337485, 2.8479314378856864, [0.222158622569474, 0.25626111181172273, 0.35182739528951196, 0.3663598873158488, 0.27742718283258855, 0.2510984653416751, 0.20495741379437485, 0.21086558444312314, 0.34637713509870655, 0.5363641061791167  …  0.7025508437981807, 0.9874553700838706, 1.266394784290792, 0.90864824841493, 0.7153302649561325, 0.4606300872849913, 0.4142989373004915, 0.575969040925019, 0.46119715365161734, 0.40109212249053455], 0.3935540555412823, 0.46857519896139493, 0.2556010203617164, [0.666666666666

In [33]:
# save(path*"metric_enkf.jld", "metric", metric_enkf)

Benchmark for the Lorenz-96 problem with the sequential stochastic radial map filter

p = 0

In [34]:
p = 0

0

In [35]:
metric_srmf0 = benchmark_srmf_lorenz96(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.772121 seconds (27.52 M allocations: 1.955 GiB, 11.79% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.377551150274605
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.749795 seconds (27.52 M allocations: 1.955 GiB, 11.57% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 1.3657938379515508
(Ne, β) = (10, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:01:04


 64.600054 seconds (1.36 G allocations: 95.559 GiB, 11.77% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:30:20


Ne 600& β 1.05 RMSE: 0.4874273248270539


8-element Array{Any,1}:
 Metrics[Metrics(10, [0.7178299117743561, 0.9938479094935131, 0.9157202489871709, 1.7712761491118478, 1.7066313005793774, 1.234897416217821, 0.9317857499383253, 0.6183698609083331, 0.7459082799620395, 1.224062120906078  …  0.6952026612202916, 0.9993289304490182, 0.3801055113850766, 0.46615893854686297, 0.4685463233371965, 0.19599471667823726, 0.19814634786104166, 0.6642794850872766, 0.9773959376079814, 0.818387844848636], 0.6979254430045315, 1.377551150274605, 2.384522230620822, [0.47066072782717444, 0.5827128366257848, 0.8457783375466058, 0.6341692687525824, 0.5343483349823486, 0.3969491055277674, 0.31329438635989737, 0.25834251139447184, 0.3398157520655201, 0.6173447950526075  …  0.7329739456388363, 0.9738238879762458, 1.1289563142068058, 0.6024237576502817, 0.43218587527191027, 0.2990092779069591, 0.2812331285358531, 0.25017593025848966, 0.41779927421383056, 0.48791228952588145], 0.6215600936267961, 0.6750599769304513, 0.29442611799893653, [0.6666666666666666

In [36]:
# save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf0)

p = 1

In [48]:
p = 1

1

In [ ]:
metric_srmf1 = benchmark_srmf_lorenz96(model, data, path, [40, 60, 100, 200, 400, 600], β_array, p);

(Ne, β) = (40, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.667425 seconds (111.02 M allocations: 8.373 GiB, 9.97% gc time)
Ne = 40
Ne 40& β 0.95 RMSE: 0.5623248141706708
(Ne, β) = (40, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.624376 seconds (110.84 M allocations: 8.353 GiB, 10.11% gc time)
Ne = 40
Ne 40& β 0.96 RMSE: 0.4917612606791061
(Ne, β) = (40, 0.97)


...

Progress: 100%|█████████████████████████████████████████| Time: 0:01:09


 69.301789 seconds (1.05 G allocations: 80.301 GiB, 10.39% gc time)
Ne = 400
Ne 400& β 0.97 RMSE: 0.3938557082299884
(Ne, β) = (400, 0.98)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:04


 64.840907 seconds (1.05 G allocations: 80.450 GiB, 10.61% gc time)
Ne = 400
Ne 400& β 0.98 RMSE: 0.4020358024647253
(Ne, β) = (400, 0.99)


Progress:  27%|███████████                              |  ETA: 0:00:46

In [ ]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf1)

p = 2

In [40]:
p = 2

2

In [41]:
metric_srmf2 = benchmark_srmf_lorenz96(model, data, path, [40, 60, 100, 200, 400, 600], β_array, p);

LoadError: [91mUndefVarError: benchmark_smrf_lorenz63 not defined[39m

In [42]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf2)

LoadError: [91mUndefVarError: metric_srmf2 not defined[39m

Benchmark for the Lorenz-96 problem with the sequential stochastic adaptive radial map filter

p = 0

In [ ]:
p = 0

In [ ]:
metric_sadaptivermf0 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 1

In [ ]:
p = 1

In [ ]:
metric_sadaptivermf1 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 2

In [ ]:
p = 2

In [ ]:
metric_sadaptivermf2 = benchmark_sadaptivermf_lorenz96(model, data, path, Ne_array, β_array, p);

In [ ]:
save(path*"metric_sadaptivermf"*string(p)*".jld", "metric", metric_sadaptivermf0)